# Set up a Locust client container

In [434]:
%%bash

export PROJECT_NAME='sandbox-kathryn-ml'
docker build -t locust_tester -f Dockerfile .
docker tag locust_tester gcr.io/${PROJECT_NAME}/locust_tester
docker push gcr.io/${PROJECT_NAME}/locust_tester

Sending build context to Docker daemon  12.31MB
Step 1/4 : FROM gcr.io/sandbox-kathryn-ml/tensorrtserver_client
 ---> c0db5114eee5
Step 2/4 : RUN pip install --upgrade locust
 ---> Using cache
 ---> 456b71165cd7
Step 3/4 : COPY locust locust
 ---> 93fd01edf7cd
Step 4/4 : COPY data data
 ---> 9a6e7f5a646a
Successfully built 9a6e7f5a646a
Successfully tagged locust_tester:latest
The push refers to repository [gcr.io/sandbox-kathryn-ml/locust_tester]
8464088aae03: Preparing
bb901fe46084: Preparing
8059cffb97be: Preparing
7c6974a457f2: Preparing
055169cf5da7: Preparing
fbd8889d27d6: Preparing
db53449549c2: Preparing
8d41da6c1def: Preparing
1e8317e242a6: Preparing
d1bb97cb5fee: Preparing
335b8eed7ca7: Preparing
2c732f48ceb6: Preparing
fb7ac298d8da: Preparing
3d1b01769526: Preparing
303059468949: Preparing
e79142719515: Preparing
aeda103e78c9: Preparing
2558e637fbff: Preparing
f749b9b0fb21: Preparing
2c732f48ceb6: Waiting
fb7ac298d8da: Waiting
d1bb97cb5fee: Waiting
3d1b01769526: Waiting
db534

## Set up a Locust Cluster

In [32]:
%%writefile service_master.yaml

kind: Service
apiVersion: v1
metadata:
  name: locust-master
  labels:
    app: locust-master
spec:
  ports:
  - name: loc-master
    port: 8089
    targetPort: 8089
    protocol: TCP
  - name: loc-master-p1
    port: 5557
    targetPort: 5557
    protocol: TCP
  - name: loc-master-p2
    port: 5558
    targetPort: 5558
    protocol: TCP
  selector:
    app: locust-master
  type: LoadBalancer

Overwriting service_master.yaml


In [33]:
!kubectl apply -f service_master.yaml

service/locust-master created


In [435]:
!kubectl get svc inference-server -o "jsonpath={.spec['clusterIP']}"

10.59.254.225

In [436]:
%%writefile deployment_master.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: locust-master
  labels:
    name: locust-master
spec:
  replicas: 1
  selector:
    matchLabels:
      app: locust-master
  template:
    metadata:
      labels:
        app: locust-master
    spec:
      containers:
      - name: locust-master
        image: gcr.io/sandbox-kathryn-ml/locust_tester
        ports:
          - name: loc-master
            containerPort: 8089
            protocol: TCP
          - name: loc-master-p1
            containerPort: 5557
            protocol: TCP
          - name: loc-master-p2
            containerPort: 5558
            protocol: TCP
        command: ["locust","-f","locust/trtis_grpc_client.py"]
        args: ["--host", "10.59.254.225", "--master"]
        resources:
          requests:
            cpu: 200m

Overwriting deployment_master.yaml


In [437]:
!kubectl get svc locust-master -o "jsonpath={.spec['clusterIP']}"

10.59.242.220

In [444]:
%%writefile deployment_slave.yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  name: locust-slave
  labels:
    name: locust-slave
spec:
  replicas: 40
  selector:
    matchLabels:
      app: locust-slave
  template:
    metadata:
      labels:
        app: locust-slave
    spec:
      containers:
      - name: locust-slave
        image: gcr.io/sandbox-kathryn-ml/locust_tester
        command: ["locust","-f","locust/trtis_grpc_client.py"]
        args: ["--slave", "--master-host=10.59.242.220"]
        resources:
          requests:
            cpu: 100m

Overwriting deployment_slave.yaml


In [449]:
!kubectl delete -f deployment_master.yaml
!kubectl delete -f deployment_slave.yaml

deployment.extensions "locust-master" deleted
deployment.extensions "locust-slave" deleted


In [446]:
!kubectl apply -f deployment_master.yaml
!kubectl apply -f deployment_slave.yaml

deployment.extensions/locust-master created
deployment.extensions/locust-slave created


In [393]:
!kubectl get svc

NAME               TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)                                        AGE
inference-server   ClusterIP      10.59.254.225   <none>          8000/TCP,8001/TCP,8002/TCP                     31h
kubernetes         ClusterIP      10.59.240.1     <none>          443/TCP                                        33h
locust-master      LoadBalancer   10.59.242.220   35.200.99.104   8089:32253/TCP,5557:30210/TCP,5558:30840/TCP   32h


In [451]:
!kubectl get pods

NAME                                READY   STATUS        RESTARTS   AGE
inference-server-56588945c6-v9kcx   1/1     Running       0          31h
locust-slave-6588496b58-47mtp       1/1     Terminating   0          36s
locust-slave-6588496b58-54qcp       0/1     Terminating   0          35s
locust-slave-6588496b58-8ln7w       1/1     Terminating   0          42s
locust-slave-6588496b58-8pxj4       0/1     Terminating   0          33s
locust-slave-6588496b58-h6z9h       1/1     Terminating   0          46s
